# Capstone-R

## Pre-requisites

### Installations

In [1]:
# Install rpy2 using pip within an R environment
system("pip install rpy2")

# Load necessary packages
install.packages("dplyr")
install.packages("evir")
install.packages(c("ggplot2", "evir", "evd", "ismev", "extRemes"))

library(dplyr)
library(readr)
library(ggplot2)
library(evir)
library(evd)
library(ismev)
library(extRemes)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppArmadillo’, ‘Lmoments’, ‘distillery’



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘evir’


The following object is masked from ‘package:ggplot2’:

    qplot



Attaching package: ‘evd’


The following objects are masked from ‘package:evir’:

    dgev, dgpd, pgev, pgpd, qgev, qgpd, rgev, rgpd


Loading required package: mgcv

Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


This is mgcv 1.9-3. For overview type 'help("mgcv-package")'.

Loading requir

### Load Relevant files

In [2]:
# Define the list of cryptocurrencies
crypto_list <- c (
    "BTC-USD", "ETH-USD", "XRP-USD", "USDT-USD",
    "ADA-USD", "DOGE-USD", "AVAX-USD", "DOT-USD",
    "LTC-USD", "TRX-USD", "LINK-USD", "BCH-USD", "XLM-USD",
    "LEO-USD", "OM-USD"
)

# Directory containing CSV files
csv_dir <- "/content/"

# List all files in the directory
csv_files <- list.files(csv_dir, pattern = "*_std_residuals.csv", full.names = TRUE)

# Match files corresponding to crypto_list
crypto_files <- csv_files[basename(csv_files) %in% paste0(crypto_list, "_std_residuals.csv")]

# Load matched CSV files into a list
crypto_data <- lapply(crypto_files, read_csv)

# Assign names to the list based on the cryptocurrency names
names(crypto_data) <- gsub("_std_residuals\\.csv$", "", basename(crypto_files))

Rows: 1458 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (1): Standardized Residuals

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1458 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (1): Standardized Residuals

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1458 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (1): Standardized Residuals

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1458 Columns: 1
── Column specification ───────────────────────────────────────

## Univariate Extremes

### Fit the GPD

In [3]:
# Define function to fit GPD
fit_gpd_threshr <- function(residuals) {
  # Ensure residuals are numeric and remove any NA values
  residuals <- as.numeric(residuals)
  residuals <- na.omit(residuals)

  # If the dataset is empty after removing NAs, skip it
  if (length(residuals) == 0) {
    message("Dataset has no valid residuals after cleaning. Skipping...")
    return(NA)
  }

  # Set the threshold at the 90th percentile
  threshold_val <- quantile(residuals, 0.90)

  # Fit the GPD using 'gpd()' from the evir package
  fit <- gpd(residuals, threshold = threshold_val)

  # Extract shape and scale parameters
  shape_val <- fit$par.ests["xi"]   # Shape parameter
  scale_val <- fit$par.ests["beta"] # Scale parameter

  # Return a list with threshold, shape, and scale parameters
  return(list(
    threshold = threshold_val,
    shape = shape_val,
    scale = scale_val
  ))
}

# Apply GPD fitting and collect results for each cryptocurrency dataset
gpd_results_threshr <- lapply(crypto_data, function(df) {
  residuals <- df$`Standardized Residuals`
  fit_gpd_threshr(residuals)
})


### The Summary Table

In [4]:
# Convert results to a structured data frame
gpd_results_df <- bind_rows(lapply(names(gpd_results_threshr), function(crypto) {
  res <- gpd_results_threshr[[crypto]]

  # Check if the result is NULL or empty
  if (is.null(res) || length(res) == 0 || all(is.na(res))) {
    return(data.frame(
      Crypto = crypto,
      Threshold = NA_real_,
      Shape = NA_real_,
      Scale = NA_real_
    ))
  }

  # Extract values for threshold, shape, and scale
  threshold_val <- ifelse(length(res$threshold) > 0, res$threshold, NA_real_)
  shape_val <- ifelse(length(res$shape) > 0, res$shape, NA_real_)
  scale_val <- ifelse(length(res$scale) > 0, res$scale, NA_real_)

  # Return structured data
  data.frame(
    Crypto = crypto,
    Threshold = threshold_val,
    Shape = shape_val,
    Scale = scale_val
  )
}), .id = NULL)  # Combine into a single data frame

# Print the structured table
print(gpd_results_df)

# Optional: Save results as CSV
write.csv(gpd_results_df, "gpd_results_threshr.csv", row.names = FALSE)


     Crypto Threshold        Shape     Scale
1   ADA-USD 1.0752835 0.1387491628 0.5688995
2  AVAX-USD 1.0673760 0.0944578842 0.5758668
3   BCH-USD 0.9368935 0.0271946218 0.6535262
4   BTC-USD 0.9946363 0.0464601187 0.7174723
5  DOGE-USD 0.9006746 0.0872031554 0.6672881
6   DOT-USD 1.0886185 0.0428915169 0.6595464
7   ETH-USD 1.0871536 0.0009239027 0.7303306
8   LEO-USD 0.9632827 0.0469471168 0.7676387
9  LINK-USD 1.1704245 0.0164791705 0.6601317
10  LTC-USD 1.0765816 0.0327991781 0.7419544
11   OM-USD 1.0274291 0.0378255182 0.6733580
12  TRX-USD 1.0224857 0.2471216806 0.5998331
13 USDT-USD 1.1947204 0.4356740296 0.4495800
14  XLM-USD 1.0561084 0.1295263530 0.6401549
15  XRP-USD 0.8583544 0.1106508326 0.5970345


## Bivaraite Extremes

### Scatter Plots of ARMA-GARCH standard residuals and Asymptotic Dependance and Independance Diagrams

In [6]:
# Generate all possible pairs of cryptocurrencies
crypto_pairs <- combn(names(crypto_data), 2, simplify = FALSE)

# Function to synchronize data lengths
trim_data <- function(df1, df2) {
  min_length <- min(nrow(df1), nrow(df2))
  df1 <- df1[1:min_length, , drop = FALSE]
  df2 <- df2[1:min_length, , drop = FALSE]
  return(list(df1 = df1, df2 = df2))
}

# Scatter Plot Function
plot_scatter_residuals <- function(df1, df2, crypto_x, crypto_y) {
  trimmed <- trim_data(df1, df2)
  df1 <- trimmed$df1
  df2 <- trimmed$df2

  combined_data <- data.frame(X = df1$`Standardized Residuals`, Y = df2$`Standardized Residuals`)

  ggplot(combined_data, aes(x = X, y = Y)) +
    geom_point(alpha = 0.6, color = "blue") +
    theme_minimal() +
    labs(title = paste("Scatter Plot:", crypto_x, "vs", crypto_y),
         x = crypto_x, y = crypto_y)
}

# Compute χ
compute_chi <- function(df1, df2, u = 0.90) {
  trimmed <- trim_data(df1, df2)
  X <- na.omit(trimmed$df1$`Standardized Residuals`)
  Y <- na.omit(trimmed$df2$`Standardized Residuals`)
  threshold_x <- quantile(X, u)
  threshold_y <- quantile(Y, u)
  exceed_x <- X > threshold_x
  exceed_y <- Y > threshold_y
  chi <- sum(exceed_x & exceed_y) / sum(exceed_x)
  return(chi)
}

# Compute χ̄
compute_chi_bar <- function(df1, df2, u = 0.90) {
  trimmed <- trim_data(df1, df2)
  X <- na.omit(trimmed$df1$`Standardized Residuals`)
  Y <- na.omit(trimmed$df2$`Standardized Residuals`)
  threshold_x <- quantile(X, u)
  threshold_y <- quantile(Y, u)
  exceed_x <- X > threshold_x
  exceed_y <- Y > threshold_y
  chi_bar <- 2 * log(mean(exceed_x & exceed_y)) / log(mean(exceed_x)) + log(mean(exceed_y))
  return(chi_bar)
}

# Plot χ vs threshold
plot_asymptotic_dependence <- function(df1, df2, crypto_x, crypto_y, u_seq = seq(0.90, 0.99, by = 0.01)) {
  chi_values <- sapply(u_seq, function(u) compute_chi(df1, df2, u))
  df_plot <- data.frame(Threshold = u_seq, Chi = chi_values)
  ggplot(df_plot, aes(x = Threshold, y = Chi)) +
    geom_line(color = "blue") +
    geom_point(color = "red") +
    theme_minimal() +
    labs(title = paste("χ Index for", crypto_x, "-", crypto_y),
         x = "Quantile Threshold", y = "Chi")
}

# Plot χ̄ vs threshold
plot_chi_bar <- function(df1, df2, crypto_x, crypto_y, u_seq = seq(0.90, 0.99, by = 0.01)) {
  chi_bar_values <- sapply(u_seq, function(u) compute_chi_bar(df1, df2, u))
  df_plot <- data.frame(Threshold = u_seq, ChiBar = chi_bar_values)
  ggplot(df_plot, aes(x = Threshold, y = ChiBar)) +
    geom_line(color = "darkgreen") +
    geom_point(color = "orange") +
    theme_minimal() +
    labs(title = paste("χ̄ Index for", crypto_x, "-", crypto_y),
         x = "Quantile Threshold", y = "Chi-bar")
}

# Initialize summary storage
summary_results <- data.frame(
  Crypto_X = character(),
  Crypto_Y = character(),
  Pearson_Correlation = numeric(),
  Chi_0.90 = numeric(),
  ChiBar_0.90 = numeric(),
  stringsAsFactors = FALSE
)

# Run everything
output_dir <- "output_plots"
dir.create(output_dir, showWarnings = FALSE)

for (pair in crypto_pairs) {
  crypto_x <- pair[1]
  crypto_y <- pair[2]

  cat("Processing:", crypto_x, "vs", crypto_y, "\n")
  df_x <- crypto_data[[crypto_x]]
  df_y <- crypto_data[[crypto_y]]

  # Save scatter plot
  scatter_plot <- plot_scatter_residuals(df_x, df_y, crypto_x, crypto_y)
  ggsave(filename = file.path(output_dir, paste0(crypto_x, "_vs_", crypto_y, "_scatter.png")),
         plot = scatter_plot)

  # Save chi plot
  ad_index_plot <- plot_asymptotic_dependence(df_x, df_y, crypto_x, crypto_y)
  ggsave(filename = file.path(output_dir, paste0(crypto_x, "_vs_", crypto_y, "_chi.png")),
         plot = ad_index_plot)

  # Save chi-bar plot
  chi_bar_plot <- plot_chi_bar(df_x, df_y, crypto_x, crypto_y)
  ggsave(filename = file.path(output_dir, paste0(crypto_x, "_vs_", crypto_y, "_chi_bar.png")),
         plot = chi_bar_plot)

  # Store correlation, chi, chi-bar at u = 0.90
  trimmed <- trim_data(df_x, df_y)
  corr <- cor(trimmed$df1$`Standardized Residuals`, trimmed$df2$`Standardized Residuals`, use = "complete.obs")
  chi_90 <- compute_chi(df_x, df_y, 0.90)
  chi_bar_90 <- compute_chi_bar(df_x, df_y, 0.90)

  summary_results <- rbind(summary_results, data.frame(
    Crypto_X = crypto_x,
    Crypto_Y = crypto_y,
    Pearson_Correlation = round(corr, 4),
    Chi_0.90 = round(chi_90, 4),
    ChiBar_0.90 = round(chi_bar_90, 4)
  ))
}

# Export summary table
write_csv(summary_results, file.path(output_dir, "crypto_dependence_summary.csv"))


Processing: ADA-USD vs AVAX-USD 


Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


Processing: ADA-USD vs BCH-USD 


Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


Processing: ADA-USD vs BTC-USD 


...

Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


Processing: USDT-USD vs XRP-USD 


Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


Processing: XLM-USD vs XRP-USD 


Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


### Visulalization

The code below is the bar plot of the coefficient of asymptotic dependence for all the pairs of cryptos

In [7]:
# Optional: Adjust for notebook output
options(repr.plot.width = 24, repr.plot.height = 24)

# Remove "-USD" from both Crypto_X and Crypto_Y
summary_results$Crypto_X_clean <- gsub("-USD", "", summary_results$Crypto_X)
summary_results$Crypto_Y_clean <- gsub("-USD", "", summary_results$Crypto_Y)

# Create 'Pair' column with cleaned names
summary_results$Pair <- paste(summary_results$Crypto_X_clean, summary_results$Crypto_Y_clean, sep = "-")

# Convert 'Chi_0.90' column to numeric (safety)
summary_results$Chi_0.90 <- as.numeric(summary_results$Chi_0.90)

# Greyscale plot for χ (Chi)
chi_plot <- ggplot(summary_results, aes(x = reorder(Pair, Chi_0.90), y = Chi_0.90)) +
  geom_col(fill = "grey", width = 0.6) +  # Greyscale bars
  coord_flip() +  # Horizontal bars
  theme_minimal(base_size = 20) +
  labs(
    title = expression("Asymptotic Dependence Index ("*Chi*") Between Cryptocurrency Pairs"),
    x = NULL,
    y = expression(Chi~"(χ) at 0.90 Quantile")
  ) +
  theme(
    plot.title = element_text(size = 22, face = "bold", hjust = 0.5),
    axis.title.y = element_blank(),
    axis.text.y = element_text(size = 20),
    axis.text.x = element_text(size = 20),
    axis.title.x = element_text(size = 20),
    plot.margin = margin(30, 30, 30, 30)
  )

# Save greyscale plot for χ
ggsave("chi_plot_greyscale.png", plot = chi_plot, width = 24, height = 24, dpi = 300)
ggsave("chi_plot_greyscale.pdf", plot = chi_plot, width = 24, height = 24)


Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on '(χ) at 0.90 Quantile' in 'mbcsToSbcs': for χ (U+03C7)”


Code below is the bar plot of the coefficient of asymptotic independence for all the pairs of cryptos

In [8]:
# Optional: Adjust for notebook output
options(repr.plot.width = 24, repr.plot.height = 24)

# Remove "-USD" from both Crypto_X and Crypto_Y
summary_results$Crypto_X_clean <- gsub("-USD", "", summary_results$Crypto_X)
summary_results$Crypto_Y_clean <- gsub("-USD", "", summary_results$Crypto_Y)

# Ensure 'Pair' column exists and is character vector
summary_results$Pair <- paste(summary_results$Crypto_X_clean, summary_results$Crypto_Y_clean, sep = "-")

# Convert 'ChiBar_0.90' column to numeric to be safe:
summary_results$ChiBar_0.90 <- as.numeric(summary_results$ChiBar_0.90)

# Greyscale plot for χ̄ (Chi-Bar) at 0.90 Quantile
chi_bar_plot <- ggplot(summary_results, aes(x = reorder(Pair, ChiBar_0.90), y = ChiBar_0.90)) +
  geom_col(fill = "grey", width = 0.6) +  # Greyscale bars
  coord_flip() +  # Horizontal bars
  theme_minimal(base_size = 20) +
  labs(
    title = expression("Asymptotic Independence Index ("*bar(Chi)*") Between Cryptocurrency Pairs at 0.90 Quantile"),
    x = NULL,
    y = expression(bar(Chi)~"(χ̄) at 0.90 Quantile")
  ) +
  theme(
    plot.title = element_text(size = 22, face = "bold", hjust = 0.5),
    axis.title.y = element_blank(),
    axis.text.y = element_text(size = 20),
    axis.text.x = element_text(size = 20),
    axis.title.x = element_text(size = 20),
    plot.margin = margin(30, 30, 30, 30)
  )

# Save greyscale plot for χ̄ at 0.90 quantile
ggsave("chi_bar_plot_0.90_greyscale.png", plot = chi_bar_plot, width = 24, height = 24, dpi = 300)
ggsave("chi_bar_plot_0.90_greyscale.pdf", plot = chi_bar_plot, width = 24, height = 24)


Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on '(χ̄) at 0.90 Quantile' in 'mbcsToSbcs': for χ (U+03C7)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on '(χ̄) at 0.90 Quantile' in 'mbcsToSbcs': for ̄ (U+0304)”


## Multivariate Extremes

To work on the Multivariate Extreme Value Analysis, we create a combined csv, and continue working in RStudio.

In [9]:
# Combine into one DataFrame (each column is one crypto)
combined_df <- as.data.frame(crypto_data)
colnames(combined_df) <- gsub("_std_residuals\\.csv$", "", basename(crypto_files))

# Save to CSV
write_csv(combined_df, "standardized_residuals_combined.csv")